In [1]:
import pandas as pd
import pickle
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import os
from pathlib import Path
from sklearn.metrics import classification_report 

In [2]:
# Set the path to the dataset
file_path = './datasets'
file_suffix = '.pkl'

In [3]:
train_malignant_local_path = f'{file_path}/train_malignant{file_suffix}'

# Load the pickle file from local
with open(train_malignant_local_path, 'rb') as f:
    train_malignant = pickle.load(f)

In [4]:
train_benign_local_path = f'{file_path}/train_benign{file_suffix}'

# Load the pickle file from local
with open(train_benign_local_path, 'rb') as f:
    train_benign = pickle.load(f)

In [5]:
# Create dataFrames for train benign and malignant images with 'images' and 'label' columns
train_malignant_df = pd.DataFrame({'images': train_malignant, 'label': 'malignant'})
train_benign_df = pd.DataFrame({'images': train_benign, 'label': 'benign'})
print(train_malignant_df.head())
print(train_benign_df.head())

                                              images      label
0  [[[44, 31, 41], [49, 36, 46], [57, 44, 54], [6...  malignant
1  [[[18, 0, 0], [108, 85, 69], [154, 131, 117], ...  malignant
2  [[[21, 19, 22], [21, 19, 22], [22, 20, 23], [2...  malignant
3  [[[80, 53, 42], [81, 54, 43], [81, 55, 42], [8...  malignant
4  [[[128, 88, 62], [153, 111, 86], [174, 130, 10...  malignant
                                              images   label
0  [[[227, 152, 172], [225, 150, 170], [222, 147,...  benign
1  [[[235, 192, 202], [235, 192, 202], [235, 192,...  benign
2  [[[114, 89, 67], [115, 90, 68], [116, 91, 69],...  benign
3  [[[195, 155, 155], [197, 157, 157], [200, 160,...  benign
4  [[[208, 175, 186], [208, 175, 186], [208, 175,...  benign


In [6]:
# Combine the dataframes
train_df = pd.concat([train_malignant_df, train_benign_df], ignore_index=True)
print(train_df.head())
print(train_df.tail())

                                              images      label
0  [[[44, 31, 41], [49, 36, 46], [57, 44, 54], [6...  malignant
1  [[[18, 0, 0], [108, 85, 69], [154, 131, 117], ...  malignant
2  [[[21, 19, 22], [21, 19, 22], [22, 20, 23], [2...  malignant
3  [[[80, 53, 42], [81, 54, 43], [81, 55, 42], [8...  malignant
4  [[[128, 88, 62], [153, 111, 86], [174, 130, 10...  malignant
                                                  images   label
11874  [[[180, 131, 134], [182, 133, 136], [182, 133,...  benign
11875  [[[165, 121, 134], [162, 118, 131], [164, 120,...  benign
11876  [[[161, 91, 101], [159, 89, 99], [154, 84, 94]...  benign
11877  [[[166, 114, 137], [167, 115, 138], [171, 119,...  benign
11878  [[[206, 178, 192], [207, 179, 193], [209, 181,...  benign


In [7]:
# Shuffle the rows 
train_df = train_df.sample(frac=1).reset_index(drop=True)
print(train_df.head())
print(train_df.tail())
print(len(train_df))

                                              images      label
0  [[[209, 119, 129], [209, 119, 129], [210, 120,...     benign
1  [[[193, 120, 131], [195, 122, 133], [197, 124,...     benign
2  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...  malignant
3  [[[123, 102, 121], [124, 103, 122], [124, 103,...  malignant
4  [[[96, 90, 58], [96, 90, 58], [97, 91, 59], [1...  malignant
                                                  images      label
11874  [[[36, 20, 21], [35, 21, 21], [36, 24, 24], [3...  malignant
11875  [[[238, 153, 150], [238, 153, 150], [237, 152,...     benign
11876  [[[21, 15, 17], [24, 18, 20], [27, 21, 23], [3...  malignant
11877  [[[197, 160, 178], [195, 158, 176], [200, 163,...     benign
11878  [[[152, 142, 140], [160, 150, 149], [168, 158,...     benign
11879


In [8]:
# Save the images to a local file
train_local_path = f"{file_path}/train{file_suffix}";

train_df.to_pickle(train_local_path)

In [9]:
test_benign_local_path = f'{file_path}/test_benign{file_suffix}'

# Load the pickle file from local
with open(test_benign_local_path, 'rb') as f:
    test_benign = pickle.load(f)

In [10]:
test_malignant_local_path = f'{file_path}/test_malignant{file_suffix}'

# Load the pickle file from local
with open(test_malignant_local_path, 'rb') as f:
    test_malignant = pickle.load(f)

In [11]:
# Create dataFrames for test benign and malignant images with 'images' and 'label' columns
test_malignant_df = pd.DataFrame({'images': test_malignant, 'label': 'malignant'})
test_benign_df = pd.DataFrame({'images': test_benign, 'label': 'benign'})
print(test_malignant_df.head())
print(test_benign_df.head())

                                              images      label
0  [[[196, 179, 195], [197, 180, 196], [199, 182,...  malignant
1  [[[149, 124, 93], [149, 124, 93], [149, 124, 9...  malignant
2  [[[159, 114, 108], [164, 119, 113], [170, 125,...  malignant
3  [[[163, 131, 136], [164, 132, 137], [166, 134,...  malignant
4  [[[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], ...  malignant
                                              images   label
0  [[[75, 40, 34], [77, 42, 36], [85, 50, 44], [8...  benign
1  [[[109, 78, 57], [108, 73, 51], [113, 74, 45],...  benign
2  [[[193, 150, 169], [190, 147, 166], [191, 148,...  benign
3  [[[44, 32, 44], [39, 26, 36], [45, 28, 38], [6...  benign
4  [[[192, 142, 151], [192, 142, 151], [193, 143,...  benign


In [12]:
# Combine the dataframes
test_df = pd.concat([test_malignant_df, test_benign_df], ignore_index=True)
print(test_df.head())
print(test_df.tail())

                                              images      label
0  [[[196, 179, 195], [197, 180, 196], [199, 182,...  malignant
1  [[[149, 124, 93], [149, 124, 93], [149, 124, 9...  malignant
2  [[[159, 114, 108], [164, 119, 113], [170, 125,...  malignant
3  [[[163, 131, 136], [164, 132, 137], [166, 134,...  malignant
4  [[[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], ...  malignant
                                                 images   label
1995  [[[155, 110, 71], [155, 110, 71], [156, 111, 7...  benign
1996  [[[37, 18, 20], [46, 28, 28], [47, 29, 29], [5...  benign
1997  [[[152, 75, 91], [149, 72, 90], [149, 72, 90],...  benign
1998  [[[18, 23, 16], [40, 43, 36], [44, 45, 39], [3...  benign
1999  [[[176, 145, 116], [176, 145, 116], [176, 145,...  benign


In [13]:
# Shuffle the rows 
test_df = test_df.sample(frac=1).reset_index(drop=True)
print(test_df.head())
print(test_df.tail())
print(len(test_df))

                                              images      label
0  [[[208, 170, 193], [209, 171, 194], [212, 174,...     benign
1  [[[121, 103, 89], [114, 96, 82], [118, 100, 88...     benign
2  [[[146, 129, 119], [151, 134, 126], [146, 128,...  malignant
3  [[[194, 152, 172], [197, 155, 175], [200, 158,...     benign
4  [[[170, 132, 131], [170, 132, 131], [170, 132,...     benign
                                                 images      label
1995  [[[200, 125, 145], [198, 123, 143], [197, 122,...  malignant
1996  [[[151, 92, 76], [152, 93, 77], [150, 91, 75],...  malignant
1997  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...  malignant
1998  [[[187, 145, 155], [191, 149, 159], [196, 154,...     benign
1999  [[[90, 51, 46], [89, 50, 45], [90, 51, 46], [9...  malignant
2000


In [14]:
# Save the images to a local file
test_local_path = f"{file_path}/test{file_suffix}";

test_df.to_pickle(test_local_path)

In [15]:
# Load the pickle file from local
train_df_read = pd.read_pickle(train_local_path)
print(train_df_read.head())
print(train_df_read.tail())
print(len(train_df_read))

                                              images      label
0  [[[209, 119, 129], [209, 119, 129], [210, 120,...     benign
1  [[[193, 120, 131], [195, 122, 133], [197, 124,...     benign
2  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...  malignant
3  [[[123, 102, 121], [124, 103, 122], [124, 103,...  malignant
4  [[[96, 90, 58], [96, 90, 58], [97, 91, 59], [1...  malignant
                                                  images      label
11874  [[[36, 20, 21], [35, 21, 21], [36, 24, 24], [3...  malignant
11875  [[[238, 153, 150], [238, 153, 150], [237, 152,...     benign
11876  [[[21, 15, 17], [24, 18, 20], [27, 21, 23], [3...  malignant
11877  [[[197, 160, 178], [195, 158, 176], [200, 163,...     benign
11878  [[[152, 142, 140], [160, 150, 149], [168, 158,...     benign
11879


In [16]:
# Load the pickle file from local
test_df_read = pd.read_pickle(test_local_path)
print(test_df_read.head())
print(test_df_read.tail())
print(len(test_df_read))

                                              images      label
0  [[[208, 170, 193], [209, 171, 194], [212, 174,...     benign
1  [[[121, 103, 89], [114, 96, 82], [118, 100, 88...     benign
2  [[[146, 129, 119], [151, 134, 126], [146, 128,...  malignant
3  [[[194, 152, 172], [197, 155, 175], [200, 158,...     benign
4  [[[170, 132, 131], [170, 132, 131], [170, 132,...     benign
                                                 images      label
1995  [[[200, 125, 145], [198, 123, 143], [197, 122,...  malignant
1996  [[[151, 92, 76], [152, 93, 77], [150, 91, 75],...  malignant
1997  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...  malignant
1998  [[[187, 145, 155], [191, 149, 159], [196, 154,...     benign
1999  [[[90, 51, 46], [89, 50, 45], [90, 51, 46], [9...  malignant
2000


In [17]:
# Separate the images and labels
X_train = train_df_read['images']
y_train = train_df_read['label']
print(X_train[0])
print(f'Shape {X_train[0].shape}')
print(y_train[0])
print(f'X type {type(X_train)}')
print(f'y type {type(y_train)}')


[[[209 119 129]
  [209 119 129]
  [210 120 130]
  ...
  [208 126 138]
  [209 129 138]
  [211 131 140]]

 [[209 119 129]
  [209 119 129]
  [210 120 130]
  ...
  [206 124 136]
  [208 128 137]
  [210 130 139]]

 [[209 119 129]
  [209 119 129]
  [209 119 129]
  ...
  [204 122 134]
  [204 124 133]
  [205 125 134]]

 ...

 [[198 117 124]
  [198 117 124]
  [197 116 123]
  ...
  [194 118 128]
  [194 121 130]
  [195 124 132]]

 [[196 115 122]
  [196 115 122]
  [195 114 121]
  ...
  [192 119 130]
  [193 120 131]
  [193 121 132]]

 [[194 113 120]
  [194 113 120]
  [193 112 119]
  ...
  [192 119 130]
  [192 119 130]
  [191 119 130]]]
Shape (224, 224, 3)
benign
X type <class 'pandas.core.series.Series'>
y type <class 'pandas.core.series.Series'>


In [18]:
# Separate the images and labels
X_test = test_df_read['images']
y_test = test_df_read['label']
print(X_test[0])
print(f'Shape {X_test[0].shape}')
print(y_test[0])
print(f'X type {type(X_test)}')
print(f'y type {type(y_test)}')

[[[208 170 193]
  [209 171 194]
  [212 174 197]
  ...
  [213 179 195]
  [212 178 194]
  [211 177 193]]

 [[209 171 194]
  [211 173 196]
  [213 175 198]
  ...
  [216 182 198]
  [215 181 197]
  [215 181 197]]

 [[212 174 197]
  [213 175 198]
  [215 177 200]
  ...
  [218 184 200]
  [218 184 200]
  [218 184 200]]

 ...

 [[187 162 183]
  [188 163 184]
  [190 165 186]
  ...
  [200 176 192]
  [198 174 190]
  [197 173 189]]

 [[186 161 182]
  [187 162 183]
  [188 163 184]
  ...
  [198 174 190]
  [195 172 188]
  [195 172 188]]

 [[185 160 181]
  [185 160 181]
  [187 162 183]
  ...
  [197 173 189]
  [195 172 188]
  [194 171 187]]]
Shape (224, 224, 3)
benign
X type <class 'pandas.core.series.Series'>
y type <class 'pandas.core.series.Series'>


In [19]:
y_train.info

<bound method Series.info of 0           benign
1           benign
2        malignant
3        malignant
4        malignant
           ...    
11874    malignant
11875       benign
11876    malignant
11877       benign
11878       benign
Name: label, Length: 11879, dtype: object>

In [20]:
# Label encode the y data
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)

In [21]:
print(type(y_train_encoded))
print(y_train_encoded[0])

<class 'numpy.ndarray'>
0


In [22]:
# Label encode the y data
y_test_encoded = encoder.transform(y_test)

In [23]:
print(type(y_test_encoded))
print(y_test_encoded[0])

<class 'numpy.ndarray'>
0


In [24]:
# Get unique shapes
X_train_shapes = [x.shape for x in X_train]
X_test_shapes = [x.shape for x in X_test]

# Get unique shapes
unique_shapes_train = np.unique(X_train_shapes, axis=0)
unique_shapes_test = np.unique(X_test_shapes, axis=0)

print("Unique shapes in X_train:", unique_shapes_train)
print("Unique shapes in X_test:", unique_shapes_test)

Unique shapes in X_train: [[224 224   3]]
Unique shapes in X_test: [[224 224   3]]


In [25]:
print(X_train[0].shape)
print(X_test[0].shape)

(224, 224, 3)
(224, 224, 3)


In [26]:
# Check the unique encoded values
unique_train_encoded, counts_train_encoded = np.unique(y_train_encoded, return_counts=True)
unique_test_encoded, counts_test_encoded = np.unique(y_test_encoded, return_counts=True)

print("Unique encoded values in y_train_encoded:", unique_train_encoded)
print("Counts of each unique value in y_train_encoded:", counts_train_encoded)
print("Unique encoded values in y_test_encoded:", unique_test_encoded)
print("Counts of each unique value in y_test_encoded:", counts_test_encoded)

Unique encoded values in y_train_encoded: [0 1]
Counts of each unique value in y_train_encoded: [6289 5590]
Unique encoded values in y_test_encoded: [0 1]
Counts of each unique value in y_test_encoded: [1000 1000]


In [27]:
# Convert the series to numpy arrays
X_train_np = np.stack(X_train.values)
X_test_np = np.stack(X_test.values)

print(X_train_np.shape) 
print(X_test_np.shape)   

(11879, 224, 224, 3)
(2000, 224, 224, 3)


In [38]:
# Define a CNN model - Model best parameters, non scaled data
model_adam_5 = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_adam_5.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 25
epochs = 10

history = model_adam_5.fit(
    X_train_np, y_train_encoded,
    validation_data=(X_test_np, y_test_encoded),
    epochs=epochs,
    batch_size=batch_size
)

Epoch 1/10
476/476 [==============================] - 54s 113ms/step - loss: 32.8549 - accuracy: 0.7411 - val_loss: 0.8382 - val_accuracy: 0.7655
Epoch 2/10
476/476 [==============================] - 50s 106ms/step - loss: 0.4508 - accuracy: 0.8461 - val_loss: 0.7692 - val_accuracy: 0.8080
Epoch 3/10
476/476 [==============================] - 50s 106ms/step - loss: 0.2860 - accuracy: 0.8976 - val_loss: 0.5710 - val_accuracy: 0.8525
Epoch 4/10
476/476 [==============================] - 51s 107ms/step - loss: 0.2202 - accuracy: 0.9214 - val_loss: 0.6400 - val_accuracy: 0.8370
Epoch 5/10
476/476 [==============================] - 51s 108ms/step - loss: 0.2550 - accuracy: 0.9166 - val_loss: 0.7059 - val_accuracy: 0.8755
Epoch 6/10
476/476 [==============================] - 52s 108ms/step - loss: 0.2132 - accuracy: 0.9326 - val_loss: 1.0174 - val_accuracy: 0.8135
Epoch 7/10
476/476 [==============================] - 50s 105ms/step - loss: 0.2343 - accuracy: 0.9316 - val_loss: 1.2257 - val_a

In [39]:
# Print the validation accuracy
val_loss, val_accuracy = model_adam_5.evaluate(X_train_np, y_train_encoded)

print(f'Validation Accuracy: {val_accuracy:.4f}')

372/372 [==============================] - 16s 43ms/step - loss: 0.1025 - accuracy: 0.9698
Validation Accuracy: 0.9698


In [40]:
# Print the validation accuracy
val_loss, val_accuracy = model_adam_5.evaluate(X_test_np, y_test_encoded)

print(f'Validation Accuracy: {val_accuracy:.4f}')

63/63 [==============================] - 3s 42ms/step - loss: 0.8934 - accuracy: 0.8680
Validation Accuracy: 0.8680


In [41]:
directory_name = 'models'

# Create a directory to save the model
if not os.path.exists(directory_name):
    os.makedirs(directory_name)
    print(f"Directory '{directory_name}' created successfully.")
else:
    print(f"Directory '{directory_name}' already exists.")

Directory 'models' already exists.


In [42]:
# Save the model
model_adam_5.save('./models/model_adam_5.h5')

In [43]:
# Set the model's file path
file_path = Path("models/model_adam_5.h5")

# Load the model to a new object
imported_adam_5 = tf.keras.models.load_model(file_path)

In [44]:
# Model summary
imported_adam_5.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 394272)            0         
                                                                 
 dense_4 (Dense)             (None, 64)                25233472  
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 25,234,433
Trainable params: 25,234,433
Non-trainable params: 0
__________________________________________

In [45]:
# Predict the test data
y_pred_unscaled = imported_adam_5.predict(X_test_np)

y_pred_unscaled = y_pred_unscaled > 0.5

# Generate classification report  
report = classification_report(y_test_encoded, y_pred_unscaled) 
print(report)

63/63 [==============================] - 3s 43ms/step
              precision    recall  f1-score   support

           0       0.86      0.89      0.87      1000
           1       0.88      0.85      0.87      1000

    accuracy                           0.87      2000
   macro avg       0.87      0.87      0.87      2000
weighted avg       0.87      0.87      0.87      2000



In [46]:
# Convert the series to numpy arrays
X_train_scaled = X_train_np / 255
X_test_scaled = X_test_np / 255

In [51]:
# Define a CNN model - Model best parameters with scaled data
model_adam_scaled = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_adam_scaled.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 25
epochs = 10

history = model_adam_scaled.fit(
    X_train_scaled, y_train_encoded,
    validation_data=(X_test_scaled, y_test_encoded),
    epochs=epochs,
    batch_size=batch_size
)

Epoch 1/10
476/476 [==============================] - 52s 109ms/step - loss: 0.6373 - accuracy: 0.7891 - val_loss: 0.3748 - val_accuracy: 0.8395
Epoch 2/10
476/476 [==============================] - 53s 110ms/step - loss: 0.3479 - accuracy: 0.8457 - val_loss: 0.4241 - val_accuracy: 0.7920
Epoch 3/10
476/476 [==============================] - 53s 112ms/step - loss: 0.3274 - accuracy: 0.8552 - val_loss: 0.4388 - val_accuracy: 0.7615
Epoch 4/10
476/476 [==============================] - 52s 109ms/step - loss: 0.3275 - accuracy: 0.8574 - val_loss: 0.3429 - val_accuracy: 0.8590
Epoch 5/10
476/476 [==============================] - 52s 110ms/step - loss: 0.3033 - accuracy: 0.8683 - val_loss: 0.3473 - val_accuracy: 0.8375
Epoch 6/10
476/476 [==============================] - 51s 107ms/step - loss: 0.2904 - accuracy: 0.8758 - val_loss: 0.3515 - val_accuracy: 0.8335
Epoch 7/10
476/476 [==============================] - 51s 108ms/step - loss: 0.2664 - accuracy: 0.8887 - val_loss: 0.3064 - val_ac

In [52]:
# Print the validation accuracy
val_loss, val_accuracy = model_adam_scaled.evaluate(X_train_scaled, y_train_encoded)

print(f'Validation Accuracy: {val_accuracy:.4f}')

372/372 [==============================] - 16s 40ms/step - loss: 0.1874 - accuracy: 0.9241
Validation Accuracy: 0.9241


In [53]:
# Print the validation accuracy
val_loss, val_accuracy = model_adam_scaled.evaluate(X_test_scaled, y_test_encoded)

print(f'Validation Accuracy: {val_accuracy:.4f}')

63/63 [==============================] - 3s 42ms/step - loss: 0.2982 - accuracy: 0.8835
Validation Accuracy: 0.8835


In [54]:
# Save the model - Scaled
model_adam_scaled.save('./models/model_adam_scaled.h5')

In [55]:
# Set the model's file path
file_path = Path("models/model_adam_scaled.h5")

# Load the model to a new object
imported_adam_scaled = tf.keras.models.load_model(file_path)

In [56]:
# Model summary
imported_adam_scaled.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 394272)            0         
                                                                 
 dense_8 (Dense)             (None, 64)                25233472  
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 25,234,433
Trainable params: 25,234,433
Non-trainable params: 0
__________________________________________

In [57]:
# Predict the test data
y_pred_scaled = imported_adam_scaled.predict(X_test_scaled)

y_pred_scaled = y_pred_scaled > 0.5

# Generate classification report  
report = classification_report(y_test_encoded, y_pred_scaled) 
print(report)

63/63 [==============================] - 3s 39ms/step
              precision    recall  f1-score   support

           0       0.86      0.92      0.89      1000
           1       0.91      0.85      0.88      1000

    accuracy                           0.88      2000
   macro avg       0.89      0.88      0.88      2000
weighted avg       0.89      0.88      0.88      2000



Grid Search for Best Parameters
Code takes approximately 13 hours to complete

In [46]:
# Parameters for Grid Search
batch_size = [25, 32]
epochs = [1, 2, 5, 10]
optimizer = ['adam', 'rmsprop', 'sgd']
parameters = {'batch_size': batch_size, 'epochs': epochs, 'optimizer': optimizer}

In [47]:
# Build model function
def build_model(optimizer):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [49]:
# Panel takes approximately 13 hours to run
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Create a new classifier to test for best parameters
new_classifier = KerasClassifier(build_fn=build_model)
grid_search = GridSearchCV(estimator=new_classifier, param_grid=parameters, scoring='accuracy', cv=10)
grid_search.fit(X_train_np, y_train_encoded)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(best_parameters)
print(best_accuracy)

/var/folders/72/_hs1tc553j3957301p31ywn00000gn/T/ipykernel_75882/2352579881.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_classifier = KerasClassifier(build_fn=build_model)


38/38 [==============================] - 2s 40ms/step
Epoch 1/2
428/428 [==============================] - 43s 100ms/step - loss: 48.9867 - accuracy: 0.5125
Epoch 2/2
38/38 [==============================] - 2s 44ms/step
Epoch 1/2
428/428 [==============================] - 43s 101ms/step - loss: 90.0891 - accuracy: 0.5070
Epoch 2/2
38/38 [==============================] - 2s 41ms/step
Epoch 1/2
428/428 [==============================] - 43s 100ms/step - loss: 109.2690 - accuracy: 0.7264
Epoch 2/2
38/38 [==============================] - 2s 40ms/step
Epoch 1/2
428/428 [==============================] - 45s 106ms/step - loss: 96.4772 - accuracy: 0.7185
Epoch 2/2
38/38 [==============================] - 2s 41ms/step
Epoch 1/2
428/428 [==============================] - 43s 100ms/step - loss: 77.8626 - accuracy: 0.6978
Epoch 2/2
38/38 [==============================] - 2s 43ms/step
Epoch 1/2
428/428 [==============================] - 42s 97ms/step - loss: 34.9645 - accuracy: 0.7337
Epoch 2/